In [2]:
import glob
# from slab import *
#import PyInventor
#from PyInventor import *
%load_ext autoreload
%autoreload 2
#import pyEPR as epr
#from pyEPR import ansys as HFSS
import numpy as np

%matplotlib notebook

import pandas as pd
import pyEPR
from pyEPR import ansys as HFSS

import scipy
from scipy import constants as const

from pint import UnitRegistry
ureg = UnitRegistry()    
Q = ureg.Quantity
import os
import matplotlib.pyplot as plt

def flt2strfmt(val, fmt='%.3f', units='mm'):
    return (fmt % val) + units

import matplotlib
import datetime
dstr = datetime.datetime.today().strftime('%y%m%d')

# Add junction and chip

In [21]:
'''
HFSS eigenmode simulation Creation:

This sets up a standard eigenmode simulation without the qubit, just the bare cavity created above. It calculates
the mode frequencies, loss (with boundary impedances set) and the electric and magnetic surface participation 
ratios (S_e, S_m)

'''
# Toggle between eigenmode and driven modal
solution_type = 'driven_modal' 
# solution_type = 'eigenmode'

# Define the cavity parameters [mm]
z_cav_1 = 100
y_cav_1 = 25
x_cav_1 = 10
z_cav_2 = 110
y_cav_2 = 25
x_cav_2 = 10
z_offset = 0
cav_thickness = 1

# Define the coupler parameters [mmm]
z_coupler = 30.
y_coupler = 0.5
x_coupler = 2.5

# Substrate dimensions
x_sub = x_coupler
y_sub = 0.450 # 450 um
z_sub = 5.

# Offsets of the junction pads
ycoupler_offset = 0.
zcoupler_offset = 0.

# Offsets of the pads from the cavity walls
pad_offset = 0.25

# Light and heavy modes (500, 100 um)
# l_junc = 500e-3
# l_junc = 100e-3
l_junc = 0.05
coupler_pads = (x_coupler - l_junc) / 2
y_pads = 0.1
w_junc = l_junc / 2 * 1e-3

# Project and HFSS design names for the bare waveguide model
dd = datetime.datetime.today().strftime('%y%m%d')
ref_name = 'planar_pads_c_coupled_three_cavities_bbq_y11_nondegen'
project_name=f'{ref_name}_{dd}'
design_name=f'{ref_name}_hfss_design_{dd}'
overwrite=True

#use file location path:
HFSS_path=os.getcwd()

full_path=HFSS_path+'\\HFSS\\'+project_name+'.aedt'

HFSS_app=HFSS.HfssApp()
HFSS_desktop=HFSS_app.get_app_desktop()

project=HFSS_desktop.open_project(full_path)

if project==None:
    project=HFSS_desktop.new_project()
    project.save(full_path)
    
project.save(full_path)
project.make_active()
    
if design_name in project.get_design_names():
    if overwrite==True:
        project.delete_design(design_name)
        project.save()
        
        # Setup a driven modal design type
        if solution_type == 'driven_modal':
            EM_design=project.new_dm_design(design_name)
        elif solution_type == 'eigenmode':
            EM_design=project.new_em_design(design_name)
        else:
            raise ValueError(f'Solution type ({solution_type}) not recognized.')
    else:
        EM_design=project.get_design(design_name)
        
else:
    if solution_type == 'driven_modal':
        EM_design=project.new_dm_design(design_name)
    elif solution_type == 'eigenmode':
        EM_design=project.new_em_design(design_name)
    else:
        raise ValueError(f'Solution type ({solution_type}) not recognized.')
        
EM_design.make_active()
model=HFSS.HfssModeler(EM_design)

# Geometry parameters
zcav1 = EM_design.set_variable('z_cav_1', flt2strfmt(z_cav_1))
ycav1 = EM_design.set_variable('y_cav_1', flt2strfmt(y_cav_1))
xcav1 = EM_design.set_variable('x_cav_1', flt2strfmt(x_cav_1))
zcav2 = EM_design.set_variable('z_cav_2', flt2strfmt(z_cav_2))
ycav2 = EM_design.set_variable('y_cav_2', flt2strfmt(y_cav_2))
xcav2 = EM_design.set_variable('x_cav_2', flt2strfmt(x_cav_2))
zcpl  = EM_design.set_variable('z_coupler', flt2strfmt(z_coupler))
ycpl  = EM_design.set_variable('y_coupler', flt2strfmt(y_coupler))
xcpl  = EM_design.set_variable('x_coupler', flt2strfmt(x_coupler))
ycpl_offset = EM_design.set_variable('ycoupler_offset',
                                    flt2strfmt(ycoupler_offset))
zcpl_offset = EM_design.set_variable('zcoupler_offset',
                                    flt2strfmt(zcoupler_offset))
z_offset  = EM_design.set_variable('z_offset', flt2strfmt(z_offset))
x_cpl_offset = EM_design.set_variable('x_cpl_offset', flt2strfmt(2e3*x_cav_1))
tcav = EM_design.set_variable('cav_thickness', flt2strfmt(cav_thickness))
cpl_pads = EM_design.set_variable('coupler_pads', flt2strfmt(coupler_pads))
ypads = EM_design.set_variable('y_pads', flt2strfmt(y_pads))
ljunc = EM_design.set_variable('l_junc', flt2strfmt(l_junc))
pdoff = EM_design.set_variable('pad_offset', flt2strfmt(pad_offset))

xsub = EM_design.set_variable('x_sub', flt2strfmt(x_sub))
ysub = EM_design.set_variable('y_sub', flt2strfmt(y_sub))
zsub = EM_design.set_variable('z_sub', flt2strfmt(z_sub))

print(f'z_offset: {z_offset}')
print(f'x_cpl_offset: {x_cpl_offset}')

# Origin
pos=[0, 0, 0]

# Draw the cavities
## Define the cavity and coupler dimensions and positions
cav1_dims = [xcav1, ycav1, zcav1]
cav2_dims = [xcav1, ycav1, zcav2]

cav1_pos = [0, -ycpl/2-ycav1/2, 0]
cav2_pos = [0, ycpl/2+ycav2/2, 0]

cpl_dims = [xcpl, ycpl, zcpl/2]
cplout_dims = [xcpl+tcav, ycpl+tcav, zcpl/2+tcav]
cpl_pos = [0, 0, 0]


# cav1 = model.draw_box_center(cav1_pos, cav1_dims, name='cavity_1')
pad1_pos = [-(cpl_pads-pdoff)/2-ljunc/2, ysub/2+ycpl_offset, zcpl_offset]
pad2_pos = [(cpl_pads-pdoff)/2+ljunc/2, ysub/2+ycpl_offset, zcpl_offset]

pad_dims = [cpl_pads-pdoff,  ypads, xcpl/10]
cpl  = model.draw_box_center(cpl_pos, cpl_dims, name='coupler')
cav1  = model.draw_box_center(cav1_pos, cav1_dims, name='cav1')
cav2  = model.draw_box_center(cav2_pos, cav2_dims, name='cav2')

model.unite(names=['coupler', 'cav1', 'cav2'],
            keep_originals=False)

# Planar pads
pad1=model.draw_rect_center(pos=pad1_pos, x_size=pad_dims[0],
                          y_size=0,
                          z_size = pad_dims[2],
                          name='pad1')
pad2=model.draw_rect_center(pos=pad2_pos, x_size=pad_dims[0],
                          y_size=0,
                          z_size = pad_dims[2],
                          name='pad2')

# Substrate
sub_pos = [0, 0, 0]
sub_dims = [xsub, ysub, zsub]
# Edit sapphire material properties
params = {'dielectric_loss_tangent' : 2e-8}
project.edit_material(params, 'sapphire')
substrate = model.draw_box_center(sub_pos, sub_dims, name='substrate',
                                 material='sapphire')
DM_setup = None

project_name: planar_pads_c_coupled_three_cavities_bbq_y11_nondegen_220821
path: Z:\tcav\ipynbs\HFSS\planar_pads_c_coupled_three_cavities_bbq_y11_nondegen_220821.aedt
z_offset: z_offset
x_cpl_offset: x_cpl_offset


## Assign the junction to be a lumped port

In [22]:
if solution_type == 'driven_modal':
    model.assign_perfect_E([pad1, pad2], name='junction_pads')
    junc_loc = [0, ysub/2+ycpl_offset, zcpl_offset]
    jj=model.draw_rect_center(pos=junc_loc, x_size=xcpl-2*cpl_pads,
                              y_size=0,
                              z_size = w_junc,
                              name='jj')

    Lj_val=10.  #<---this sets the nominal Lj_val, should be same as start
    L_jj=EM_design.set_variable('Lj_1', '%.3fnH'%Lj_val)
    jj_line=model.draw_polyline(jj.make_center_line(axis='X'), 
                                closed=False, name='jj_line')
    jj.make_lumped_port('X', z0=1e6)

## Interpolating sweep to identify resonances

In [23]:
f1 = 10.; f2 = 20.; count = 20001

In [24]:
# Define the sweep range
if not DM_setup:
    DM_setup = EM_design.create_dm_setup()
DM_setup.insert_sweep(f1, f2,
                      count=count,
                      save_fields=True,
                      name=f'Interpolating_0',
                      type='Interpolating')

DM_setup.solve()

0

In [25]:
def set_axes_fonts(ax, fsize):
    """
    Set axes font sizes because it should be abstracted away
    """
    for tick in ax.get_xticklabels():
        tick.set_fontsize(fsize)
    for tick in ax.get_yticklabels():
        tick.set_fontsize(fsize)

def set_xaxis_rot(ax, angle=45):
    """
    Rotate the x-axis labels
    """
        
    for tick in ax.get_xticklabels():
        tick.set_rotation(angle)

def set_leg_outside(ax, fsize):
    """
    Sets the legend location outside
    """
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    
    # Put a legend to the right of the current axis
    hdls, legs = ax.get_legend_handles_labels()
    leg = ax.legend(hdls, legs, loc='center left', bbox_to_anchor=(1, 0.5), \
                    fontsize=fsize, framealpha=0.)

    return leg

### Get the interpolating sweep data to estimate resonance positions

In [26]:
fsweep_interp = HFSS.HfssFrequencySweep(DM_setup, 'Interpolating_01')

report_im = fsweep_interp.create_report('Interpolating_01', expr='im(Y(LumpPort,LumpPort))')
yim = report_im.get_arrays()

# Identify the zero crossings as resonances
zero_xing_idxs = np.where(np.diff(np.sign(yim[1,:])))[0]
resonances = yim[0,zero_xing_idxs]

com_error: (-2147417851, 'The server threw an exception.', None, None)

In [ ]:
resonances

In [ ]:
# Plot the results, save the figure(s)
fsize = 20
fscale = 1e9
#wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gD, chippp = \
#get_wp_Cp_Lp(fdata, ydata, LJ0=None, CJ0=None, fscale=1e9, use_Zq=False)

fig, ax = plt.subplots(1, 1, tight_layout=True)
ax.plot(yim[0,:], yim[1,:], '-', label='Zero Crossings')
ax.plot(resonances, yim[1,zero_xing_idxs], 'o', label='Zero Crossings')
# ax.plot(fdata, ydata.imag)
ax.set_ylabel(r'Im $Y(\omega)$', fontsize=fsize)
ax.set_xlabel(r'Frequency [GHz]', fontsize=fsize)
set_axes_fonts(ax, fsize)

#### Bypass the interpolating sweep if frequencies already known

In [ ]:
# resonances = np.array([ 6.0928,  6.1684,  6.1692,  6.2692,  7.3356,  7.3816,  7.4712,
#        7.506 ,  9.3064,  9.3292,  9.588 ,  9.6076, 11.5804, 11.6684,
#       11.7568, 11.95  , 12.604 , 12.6944, 12.8012, 12.8048])
# resonances = np.array([ 6.0928,  6.1692,  6.2692,  7.3816,  7.4712,
#        7.506 , 9.3292,  9.588 ,  9.6076, 11.5804, 
#       11.7568, 11.95, 12.6944, 12.8048])

In [ ]:
# Use half the minimum spacing between resonances as a the bandwidth

In [ ]:
df = 0.8 * np.diff(np.hstack((f1, resonances, f2))).min()

### Discrete sweep to resolve resonances

In [ ]:
if solution_type == 'driven_modal':
    # Setup the driven-modal simulation
    if not DM_setup:
        DM_setup = EM_design.create_dm_setup()
        
    # Discrete count
    count = 201
    
    ## This should resolve the issue of overlapping sweeps
    ## We advance the left boundary by a small amount away
    ## from the previous right boundary
    dff = df / count
    
    # Identify resonances from interpolating sweep manually (for now)
    ## Start at the first sweep frequency
    f1r = f1
    
    # df  = 0.01 # 10 MHz window
    # Iteratve over all resonances and shift the discrete
    # sweep window from f1r to f2r
    rlen = len(resonances)
    fdata = np.array([])
    ydata = np.array([])
    for ridx, fr in enumerate(resonances):
        try:
            # First sweep from f1 to fr_j - df / 2
            f2r = fr - df / 2
            print(f'[{2*ridx} / {2*rlen}] Discrete sweep from {f1r} to {f2r} GHz ...')
            DM_setup.insert_sweep(f1r, f2r,
                                  count=count,
                                  save_fields=True,
                                  name=f'Discrete_Sweep_{2*ridx}',
                                  type='Discrete')
            DM_setup.solve()

            # Get data from first sweep
            print(f'Creating even reports ...')
            fsweep = HFSS.HfssFrequencySweep(DM_setup, f'Discrete_Sweep_{2*ridx}')
            report_re = fsweep.create_report(f'Discrete_Sweep_{2*ridx}', expr='re(Y(LumpPort,LumpPort))')
            report_im = fsweep.create_report(f'Discrete_Sweep_{2*ridx}', expr='im(Y(LumpPort,LumpPort))')
            yre = report_re.get_arrays()
            yim = report_im.get_arrays()
            fdata = np.hstack((fdata, yre[0,:]))
            ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))    

            # Backup data to file
            print(f'Backing up even reports ...')
            with open(f'../data/ydata_{dd}_sweep_{2*ridx}.csv', 'w') as fid:
                fid.write('\n'.join([f'{ff}, {yi}, {yr}'\
                                     for ff, yi, yr in zip(yre[0,:], yre[1,:], yim[1,:])]))

            # Second sweep from fr_j - df / 2 to fr_j + df / 2
            f1r = f2r + dff
            f2r = fr + df / 2
            print(f'[{2*ridx+1} / {2*rlen}] Discrete sweep from {f1r} to {f2r} GHz ...')
            DM_setup.insert_sweep(f1r, f2r,
                                  count=count,
                                  save_fields=True,
                                  name=f'Discrete_Sweep_{2*ridx+1}',
                                  type='Discrete')
            DM_setup.solve()

            # Get data from first sweep
            print(f'Creating odd reports ...')
            fsweep = HFSS.HfssFrequencySweep(DM_setup, f'Discrete_Sweep_{2*ridx+1}')
            report_re = fsweep.create_report(f'Discrete_Sweep_{2*ridx+1}', expr='re(Y(LumpPort,LumpPort))')
            report_im = fsweep.create_report(f'Discrete_Sweep_{2*ridx+1}', expr='im(Y(LumpPort,LumpPort))')
            yre = report_re.get_arrays()
            yim = report_im.get_arrays()
            fdata = np.hstack((fdata, yre[0,:]))
            ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))    

            # Backup data to file
            print(f'Backing up even reports ...')
            with open(f'../data/ydata_{dd}_sweep_{2*ridx+1}.csv', 'w') as fid:
                fid.write('\n'.join([f'{ff}, {yi}, {yr}'\
                                     for ff, yi, yr in zip(yre[0,:], yre[1,:], yim[1,:])]))

            # Next sweep from fr_j + df / 2 to fr_j+1 - df / 2
            f1r = f2r + dff
    
        except Exception as e:
            print(f'Exception on run {ridx}:\n{e}')
            continue
    
    # Last frequency band from fr_j + df / 2 to f2
    f2r = f2
    print(f'[{2*rlen} / {2*rlen}] Discrete sweep from {f1r} to {f2r} GHz ...')
    DM_setup.insert_sweep(f1r, f2r,
                          count=count,
                          save_fields=True,
                          name=f'Discrete_Sweep_{2*rlen}',
                          type='Discrete')
    DM_setup.solve()
    
    # Get data from first sweep
    print(f'Creating last report ...')
    fsweep = HFSS.HfssFrequencySweep(DM_setup, f'Discrete_Sweep_{2*rlen}')
    report_re = fsweep.create_report(f'Discrete_Sweep_{2*rlen}', expr='re(Y(LumpPort,LumpPort))')
    report_im = fsweep.create_report(f'Discrete_Sweep_{2*rlen}', expr='im(Y(LumpPort,LumpPort))')
    yre = report_re.get_arrays()
    yim = report_im.get_arrays()
    fdata = np.hstack((fdata, yre[0,:]))
    ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))

    # Backup data to file
    print(f'Backing up even reports ...')
    with open(f'../data/ydata_{dd}_sweep_{2*rlen}.csv', 'w') as fid:
        fid.write('\n'.join([f'{ff}, {yi}, {yr}'\
                             for ff, yi, yr in zip(yre[0,:], yre[1,:], yim[1,:])]))

In [ ]:
fdata

In [ ]:
ydata

In [ ]:
fdata.size

In [ ]:
ydata.size

## Get admittance data from file, if above failed

In [20]:
recover_from_files = False
if recover_from_files:
    fdata = np.array([])
    ydata = np.array([], dtype=complex)

    # Iterate over all discrete frequency sweeps
    for i in range(2*len(resonances)+1):
        report_re = fsweep_interp.create_report(f'Discrete_Sweep_{i}', expr='re(Y(LumpPort,LumpPort))')
        report_im = fsweep_interp.create_report(f'Discrete_Sweep_{i}', expr='im(Y(LumpPort,LumpPort))')
        yre = report_re.get_arrays()
        yim = report_im.get_arrays()

        fdata = np.hstack((fdata, yre[0,:]))
        ydata = np.hstack((ydata, yre[1,:]+1j*yim[1,:]))

In [21]:
fdata.size

5025

## Combine data from several discrete sweeps

In [22]:
def combine_multiple_data(files : list, Nzeros : int=10):
    """
    Combines data from multiple sweeps, fills with zeros outside of run
    """
    # Iterate over all files
    zero_pad = np.ones(Nzeros)
    fdata = np.array([])
    yredata = np.array([])
    yimdata = np.array([])
    data0 = np.genfromtxt(files[0], delimiter=',', skip_header=1).T
    
    if len(files) > 1:
        for fidx, file in enumerate(files[1:]):

            # Get the current data file
            data1 = np.genfromtxt(file, delimiter=',', skip_header=1).T
            
            # Store the previous data
            fdata = np.hstack((fdata, data0[0]))
            yredata = np.hstack((yredata, data0[1]))
            yimdata = np.hstack((yimdata, data0[2]))
            
            # Determine the frequency array and zeros to append
            #fapp = np.linspace(data0[0,-1], data1[0,0], Nzeros)
            #print(f'fapp: [{fapp[0]/1e9}, {fapp[-1]/1e9}] GHz')
            #fdata = np.hstack((fdata, fapp))
            #yredata = np.hstack((yredata, zero_pad * yredata[-1]))
            #yimdata = np.hstack((yimdata, zero_pad * yimdata[-1]))
            data0 = np.copy(data1)
            
        # Store the last data set
        fdata = np.hstack((fdata, data0[0]))
        yredata = np.hstack((yredata, data0[1]))
        yimdata = np.hstack((yimdata, data0[2]))
        
        # Convert to numpy arrays and make y complex
        fdata = np.asarray(fdata).flatten()
        ydata = yredata.flatten() + 1j * yimdata.flatten()
        
        return fdata, ydata

In [23]:
#files = [f'../data/ydata_220801_sweep_{d}.csv' for d in range(25)]
#fdata, ydata = combine_multiple_data(files)

## Postprocessing, blackbox quantization

In [ ]:
def get_wp_Cp_Lp(f, y, LJ0=10e-9, CJ0=2e-15, RJ0=10e3, fscale=1e9, use_Zq=False):
    """
    Converts frequency-dependent imaginary parts
    of the admittance data to poles,
    capacitances, and inductances
    """
    # Add the junction inductance and capacitance to the
    # total admittance
    # Compute the zero crossings
    fcpy = np.copy(f * fscale)
    ycpy = np.copy(np.complex128(y))
    if LJ0:
        ycpy += (1. / (1j * fcpy * 2 * np.pi * LJ0)) \
          + 1j * fcpy * 2 * np.pi * CJ0 \
          + (1. / RJ0)
    
    yim = ycpy.imag
    zero_xing_idxs = np.where(np.diff(np.sign(yim)))[0]
    wp = 2 * np.pi * fcpy[zero_xing_idxs]
    yp = yim[zero_xing_idxs]

    # Compute the capacitances from the derivatives
    ypp = np.gradient(ycpy, 2*np.pi*(fcpy[1]-fcpy[0]))
    Cp = 0.5 * np.abs(ypp.imag[zero_xing_idxs])
    Lp = 1. / (wp**2 * Cp)
    Zp = np.sqrt(Lp / Cp)
    Rp = 1. / np.abs(np.real(ycpy[zero_xing_idxs]))
    Qp = wp * Rp * Cp
    
    # Physical constants
    h    = 6.62607015e-34 
    hbar = h / (2 * np.pi)
    e    = 1.602176634e-19
    Phi0 = h / (2 * e)
    
    # Compute Ej, Ec
    if LJ0 and CJ0 and RJ0:
        # Compute the Josephson inductive and charging energies
        Ej = (Phi0 / (2 * np.pi))**2 / LJ0
        Ec = e**2 / (2 * CJ0)

        # Compute inductive energies, EL
        EL = 0.5 * Phi0**2 / Lp

        # Compute the phase zero point fluctuations
        GQ = 2 * e**2 / h
        phip2 = 2 * np.pi * GQ * Zp

        # Compute the participations
        # p = phi2 * 2 * Ej / (hbar * wp)
        pl = Lp / LJ0
        pc = Cp / CJ0

        # Compute the gs / Ds
        Zq = np.sqrt(LJ0 / CJ0)
        gD = np.sqrt(Zp / Zq)
        
        # Compute the g's in units of Hz, divide by h to get v, Ej = hv
        gpq = -(2 * np.pi * GQ * Ej) * (0.5 - sum(phip2) * np.asarray([[
            np.sqrt(Zpp * Zqq) / h for p, Zpp in enumerate(Zp)]
                               for q, Zqq in enumerate(Zp) ]) / 24)
        gpq = gpq.reshape([len(gpq), len(gpq)])
        
        # Compute the chis, anharmonicities
        # In units of Hz, divide by h to get v, Ec = hv
        chipp  = -Ec * gD**2 / h
        chippp = np.asarray([-2 * np.sqrt(chipp[p] * chipp[q]) 
                             for p in range(len(chipp))
                             for q in range(len(chipp))])
        chippp = chippp.reshape([len(chipp), len(chipp)])

        return wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gpq, chippp
    
    else:
        return wp, Lp, Cp, Qp, fcpy, ycpy, yp, None, None, None, None

## Plot the zero crossings

In [ ]:
# Plot the results, save the figure(s)
fsize = 20
fscale = 1e9
wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gD, chippp = \
get_wp_Cp_Lp(fdata, ydata, LJ0=None, CJ0=None, fscale=1e9, use_Zq=False)

fig, ax = plt.subplots(1, 1, tight_layout=True)
ax.plot(fdata, ydata.imag)
ax.plot(wp/(2*np.pi*fscale), yp.imag, 'o', label='Zero Crossings')
ax.set_ylabel(r'Im $Y(\omega)$', fontsize=fsize)
ax.set_xlabel(r'Frequency [GHz]', fontsize=fsize)
set_axes_fonts(ax, fsize)
#ax.set_xlim([6.08, 6.081])
# fig.savefig(f'../figs/zeros_term_bbq_inductive_50um_{dstr}.pdf', format='pdf')

## Iterate over all coupler offset values, fix LJ=10 nH

In [ ]:
NLJ = 25
wps = []
pcs = []
pls = []
gDs = []
Cps = []
Lps = []
Qps = []
chis = []
Nmodes = 10

# Whether or not to use the largest impedance mode
# as the junction mode in the calculations
use_Zq = False
print_results = True

# LJs = np.logspace(np.log10(0.1e-9), np.log10(1000e-9), NLJ)
LJs = np.logspace(np.log10(1e-9), np.log10(40e-9), NLJ)
# LJs = np.logspace(np.log10(0.00001e-9), np.log10(0.0001e-9), NLJ)
# LJs = np.logspace(np.log10(0.1), np.log10(100), NLJ)
LJ0 = 10e-9
#for idx, x in enumerate(xdata):
for LJ in LJs:
    wpp, Lp, Cp, Qp, _, _, _, pl, pc, gD, chip \
    = get_wp_Cp_Lp(fdata, ydata, LJ0=LJ, fscale=fscale, use_Zq=use_Zq)
    if print_results:
        print(f'wpp: {wpp}')
        #print(f'Lp: {Lp}')
        #print(f'Cp: {Cp}')
        #print(f'pl: {pl}')
        #print(f'pc: {pc}')
        #print(f'chip: {chip}')
        print(f'g: {gD}')
    # wps.append(np.sort(wpp[0:Nmodes]))
    wps.append(wpp[0:Nmodes])
    pls.append(pl[0:Nmodes])
    pcs.append(pc[0:Nmodes])
    gDs.append(gD[0:Nmodes, 0:Nmodes])
    Lps.append(Lp[0:Nmodes])
    Cps.append(Cp[0:Nmodes])
    Qps.append(Qp[0:Nmodes])
    chis.append(chip[0:Nmodes, 0:Nmodes])
    # wps.append(wpp)
    # print(f'LJ: {LJ}, wpp: {wpp}')
wps = np.asarray(wps).T
pls  = np.asarray(pls).T
pcs  = np.asarray(pls).T
gDs = np.asarray(gDs).T
Lps = np.asarray(Lps).T
Cps = np.asarray(Cps).T
Qps = np.asarray(Qps).T
chis = np.asarray(chis).T

In [ ]:
wp0,_ , _, _, _, _, _, _, _, _, _ = get_wp_Cp_Lp(fdata, ydata, fscale=fscale, LJ0=None)

## Plot the modes of coupled structure

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
for idx, wppp in enumerate(wps):
    ax.plot(LJs/1e-9, wppp/(2*np.pi*fscale),
            'o', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
    ax.plot(LJs/1e-9, [wp0[idx]/(2*np.pi*fscale)] * len(LJs),
            '--', color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
# ax.set_ylim([6.158, 6.166])
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
fig.savefig(f'../figs/modes_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

In [ ]:
dw

In [ ]:
wps[0]

## Plot the splittings of the lowest modes vs LJ

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
ax.plot(LJs/1e-9, (wps[1]-wps[0])/(2*np.pi*fscale2),
           'o-',
            color=cycle[0%clen])
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Mode Splitting (0-1) [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
# ax.set_ylim([0, 100])
# ax.set_xlim([1, 2.2])
ax.set_xscale('log')
fig.savefig(f'../figs/modes_splitting_term_bbq_inductive_planar_sapphire_{dstr}.pdf',
            format='pdf')

## Mode losses

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
for idx, Q in enumerate(Qps):
    ax.plot(LJs/1e-9, 1./Q,
            'o-', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$Q_p^{-1}$', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_losses_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

## Black box results

### $\alpha_p$ and $\chi_{pq}$

In [ ]:
chis.shape

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
for idx in range(Nmodes):
    ax.plot(LJs/1e-9, 0.5 * np.abs(chis[idx, idx, :]) / fscale2,
            'o-', label=r'Mode-%d' % (idx + 1),
            color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$|\alpha_p|$ [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_alpha_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

In [ ]:
# Chi, lower triangle
chiT = np.array([np.triu(chis[:,:,i]) for i in range(len(LJs))]).T
gT = np.array([np.triu(gDs[:,:,i]) for i in range(len(LJs))]).T

In [ ]:
chiT[0:NM,0:NM,0]

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
NM = 5
idx = 0
for i in range(NM):
    for j in range(NM):
        if (i != j) and np.any(chiT[i, j, :] != 0.):
            ax.plot(LJs/1e-9, np.abs(chiT[i, j, :]) / fscale2,
                'o-', label=r'$\chi_{%d,%d}$' % (i + 1, j + 1),
                color=cycle[idx%clen])
            idx += 1
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$|\chi_{pq}|$ [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_chi_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
NM = 5
idx = 0
for i in range(NM):
    for j in range(NM):
        if (i != j) and np.any(gT[i, j, :] != 0.):
            ax.plot(LJs/1e-9, np.abs(gT[i, j, :]) / fscale2,
                'o-', label=r'$g_{%d,%d}$' % (i + 1, j + 1),
                color=cycle[idx%clen])
            idx += 1
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$|g_{pq}|$ [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/mode_g_term_bbq_inductive_planar_sapphire_{dstr}.pdf', format='pdf')